# Parasitic (Zero-lift) drag

This section describes how to compute parasitic (zero-lift) drag coefficient ($C_{D_0}$) using drag buildup method and demonstrates it using the example airplane. The drag buildup method consists of computing zero-lift drag for each major component of the airplane and then summing it up to obtain total zero-lift drag for the entire airplane. The zero lift drag for any component can be divided into two parts: skin friction drag and form drag. The skin friction drag is calculated based on the equivalent flat plate theory. However, in conceptual design phase, it is difficult to get a good estimate of form drag. Hence, a factor (greater than 1) is multiplied to skin friction drag to account for form drag. Once parasitic drag for individual components are evaluated, they are added to obtain final value. Additionally, extra terms representing other sources of zero-lift drag can also be added. Refer to lecture notes, section 12.5 in Raymer, section 13.3 in Nicolai and Carichner, and section 15.4.6 in Gudmundsson for more details. Mathematically, the drag buildup method can be written as

$$
    C_{D_0} = \frac{ \sum\limits_{i=1}^n C_{f_i} FF_i Q_i S_{wet_i} }{ S_{ref}} + C_{D_{misc}} + C_{D_{L\&P}},
$$

where $C_f$ is the skin friction coefficient, $FF$ represents form factor to account for form drag, $Q_i$ denotes inteference factor, $S_{wet}$ is the wetted area, and $S_{ref}$ is the reference area. The subscript $i$ denotes $i^{th}$ component and $n$ is the total number of components. The $C_{D_{misc}}$ represents miscellaneous drag caused by flaps, fixed landing gear, fuselage upsweep angle, etc. The $C_{D_{L\&P}}$ refers to the drag due to leakages and protuberances.

For the example airplane, four components are considered in clean (cruise) configuration: wing, fuselage, horizontal tail, and vertical tail. Below subsections compute each term in the above equation, and then computes final $C_{D_0}$ for the example airplane.

## Skin friction coefficient

The skin friction coefficient ($C_f$) is computed by assuming flow over a flat plate. The value of $C_f$ is dependent on the type of flow (laminar or turbulent), which in turn is dependent on the Reynolds number ($Re$), given by

$$
    Re = \frac{\rho Vl}{\mu},
$$

where $\rho$ is the density, $V$ is the freestream velocity, $l$ is the characteristic length (typically length of the component in the direction of flow), and $\mu$ is the dynamic viscosity. Based on the $Re$, $C_f$ is computed as (Raymer Eq. 12.26 and 12.27):

$$
    C_f = \frac{1.328}{\sqrt{Re}} \text{ (laminar)} \quad \text{and} \quad C_f = \frac{0.455}{(\log_{10}Re)^{2.58}} \text{ (turbulent)} 
$$

Usually, the flow over airplane components is almost turbulent. Table 12.4 in Raymer provides some guidance for estimating what percent of wetted area might be experiencing laminar flow. For the example airplane, it is assumed that fuselage has 10% laminar flow, while all other components have fully turbulent flow. If the flow is mixed, then $C_f$ is computed as a weighted average of laminar and turbulent values.

Note that skin roughness plays an important role in skin friction drag. As recommended by Raymer, a cutoff Re (equation 12.28 in Raymer) can be used to account for the increased turbulent flow. It was determined that the cutoff Re was much large than the actual Re for the example airplane, and hence, not used here. Additionally, $C_f$ for turbulent flow can be corrected for compressibility effects using Raymer equation 12.27, but it was not used since example airplane will be flying at low speeds.

Below code block computes skin friction coefficient for each component:

In [1]:
import math

def compute_re_cruise(l):
    """
        Function to compute Re, l should be in m
    """

    rho = 0.962870 # kg/m^3, 8000 ft
    mu = 0.0000173188 # Pa.s, 8000 ft
    V = 102.89 # m/s

    return rho*V*l/mu

compute_cf_laminar = lambda re: 1.328/re**0.5 # Cf for turbulent flow
compute_cf_turbulent = lambda re: 0.455/math.log10(re)**2.58 # Cf for laminar flow

# Compute Re
fuselage_re = compute_re_cruise(31.6*0.3048) # 31.6 ft to m
wing_re = compute_re_cruise(4.3*0.3048) # 4.3 ft to m
ht_re = compute_re_cruise(2.4*0.3048) # 2.4 ft to m
vt_re = compute_re_cruise(3.7*0.3048) # 3.7 ft to m

# Compute fuselage Cf
Cf_fuselage_laminar = compute_cf_laminar(fuselage_re)
Cf_fuselage_turbulent = compute_cf_turbulent(fuselage_re)
Cf_fuselage = Cf_fuselage_laminar*0.1 + Cf_fuselage_turbulent*0.9
print(f"Cf fuselage: {Cf_fuselage:.6f}")

# Compute wing Cf
Cf_wing = compute_cf_turbulent(wing_re)
print(f"Cf wing: {Cf_wing:.6f}")

# Compute ht Cf
Cf_ht = compute_cf_turbulent(ht_re)
print(f"Cf ht: {Cf_ht:.6f}")

# Compute vt Cf
Cf_vt = compute_cf_turbulent(vt_re)
print(f"Cf vt: {Cf_vt:.6f}")

Cf fuselage: 0.002103
Cf wing: 0.003147
Cf ht: 0.003467
Cf vt: 0.003225


## Form and Interference factors

Next, form factor ($FF$) for each component is determined. The $FF$ for wings and tails is determined using:

$$
    FF = \bigg[ 1 + \frac{0.6}{(x/c)_m} (t/c) + 100 (t/c)^4 \bigg] [ 1.34 M^{0.18} (\cos\Lambda_m)^{0.28} ] ,
$$

where $(x/c)_m$ is the location of maximum thickness as a fraction of chord, $t/c$ is the thickness-to-chord ratio, and $\Lambda_m$ is the sweep along maximum thickness.

For the example airplane, $(x/c)_m = 0.3$ for the wing and tails. The $t/c$ is set to 0.18 for the wing and 0.12 for the tails. The form factor computed using above equation can be adjusted for compressibility and sweep effects, but it was not done here since the example airplane will be flying at low speeds and has no sweep. The $FF$ for fuselage is determined using:

$$
    FF = \bigg[ 0.9 + \frac{5}{f^{1.5}} + \frac{f}{400} \bigg],
$$

where $f$ is the fuselage fineness ratio. The interference factor ($Q$) for fuselage and wing is set to 1.0, while it is set to 1.04 for both tails. Refer to section 12.5.4 and 12.5.5 in Raymer for more details about $FF$ and $Q$.

Below code block computes $FF$ for wing, fuselage, horizontal tail, and vertical tail:

In [2]:
M = 0.3 # cruise mach number
compute_ff_fuselage = lambda f: 0.9 + 5/f**1.5 + f/400
compute_ff_wing_tail = lambda xc, tc, sweep: (1 + 0.6*tc/xc + 100*tc**4) * (1.34*M**0.18*math.cos(sweep*math.pi/180)**0.28)

ff_fuselage = compute_ff_fuselage(5.5)
ff_wing = compute_ff_wing_tail(0.3, 0.18, 0)
ff_ht = compute_ff_wing_tail(0.3, 0.12, 10)
ff_vt = compute_ff_wing_tail(0.3, 0.12, 30)

print(f"Fuselage form factor: {ff_fuselage:.6f}")
print(f"Wing form factor: {ff_wing:.6f}")
print(f"Horizontal tail form factor: {ff_ht:.6f}")
print(f"Vertical tail form factor: {ff_vt:.6f}")

Fuselage form factor: 1.301388
Wing form factor: 1.580582
Horizontal tail form factor: 1.354407
Vertical tail form factor: 1.306530


## Total Skin friction drag

Once $C_f$, $FF$, and $Q$ are known for each component, skin friction component of the parasitic drag can be calculated. Note that the wetted area for each component is obtained from the OpenVSP models. The fuselage and wing have a wetted area of 380 $\text{ft}^2$ and 224 $\text{ft}^2$, respectively. The horizontal and vertical tail have a wetted area of 60 $\text{ft}^2$ and 39 $\text{ft}^2$, respectively.

Below code block computes skin friction component of the parasitic drag:

In [3]:
Sref = 134 # sq ft
CD0_sf = 0

# Fuselage
Q_fuselage = 1.0
S_wet_fuselage = 380 # sq ft
CD0_sf += Cf_fuselage * ff_fuselage * Q_fuselage * S_wet_fuselage / Sref

# Wing
Q_wing = 1.0
S_wet_wing = 224 # sq ft
CD0_sf += Cf_wing * ff_wing * Q_wing * S_wet_wing / Sref

# Horizontal tail
Q_ht = 1.04
S_wet_ht = 60 # sq ft
CD0_sf += Cf_ht * ff_ht * Q_ht * S_wet_ht / Sref

# Vertical tail
Q_vt = 1.04
S_wet_vt = 39 # sq ft
CD0_sf += Cf_vt * ff_vt * Q_vt * S_wet_vt / Sref

print(f"Skin friction drag contribution to parasitic drag: {CD0_sf:.6f}")

Skin friction drag contribution to parasitic drag: 0.019538


## Miscellaneous drag

The miscellaneous drag usually involves drag due to flaps, landing gear, non-smooth external stores, fuselage upsweep angle, etc. In this section, only drag due to fuselage upsweep angle is considered since example airplane does not contain any external stores. The drag to flaps and landing gear is not included since only clean (cruise) configuration drag is addressed in this section.

The drag due to fuselage upsweep angle is computed using (Raymer equation 12.36):

$$
    (D/q)_{\text{upsweep}} = 3.83u^{2.5}A_{max},
$$

where $u$ is the fuselage upsweep angle (in radians), $A_{max}$ is the maximum cross-sectional area of fuselage, and $D/q$ is the drag area. Dividing $D/q$ by $S_{ref}$ yields parasitic drag coefficient. Similarly, $D/q$ for windshield is obtained from Table 12.6 in Raymer. Note that the $D/q$ for the windshield should be multiplied by the ratio of frontal area of windshield and reference area to obtain component of parasitic drag coefficient.

Since example airplane is twin-engine propeller driven, one has to compute drag from propellers during an engine out scenario. Since variable pitch propellers are used, it can be feathered so that mid-to-outer section of the propeller is along the airflow and creates minimal drag. The drag from a feathered propeller can be computed as:

$$
    (D/q)_{\text{feathered prop}} = 0.1\sigma A_{\text{propeller disk}}
$$

Note that the drag from a feathered propeller will be included in $C_{D_{misc}}$ only during an engine out scenario only.

Below block computes $C_{D_{misc}}$ for the example airplane:

In [4]:
Sref = 134 # sq ft
CDmisc = 0

# Fuselage upsweep drag
u = 10 * math.pi / 180 # rad
Amax = 24 # sq ft - from fusealge cross-section
D_by_q_upsweep = 3.83 * u**2.5 * Amax

CDmisc += D_by_q_upsweep / Sref

# Windshield drag
D_by_q_windshield =  0.07
CDmisc += D_by_q_windshield * 0.3 * Amax / Sref

print(f"Miscellaneous drag contribution to parasitic drag: {CDmisc:.6f}")

# Feathered propeller
sigma = 0.786
A_prop = math.pi * 5.8**2 / 4 # sq ft
D_by_q_prop = 0.1*sigma*A_prop

CDmisc_oei = CDmisc + D_by_q_prop / Sref # one engine inoperative

print(f"Miscellaneous drag during one engine inoperative: {CDmisc_oei:.6f}")

Miscellaneous drag contribution to parasitic drag: 0.012491
Miscellaneous drag during one engine inoperative: 0.027988


## Final zero-lift drag

Once all the requried terms are computed, final value of $C_{D_0}$ can be computed. Note that the leakage and protuberance drag is assumed to be 5% of the total drag. Below block computes the $C_{D_0}$ for the example airplane:

In [5]:
CD0 = CD0_sf + CDmisc

CD0 *= 1.05 # Leakage and protuberance drag assumed to be 5% of total drag

print(f"Final zero-lift drag coefficient, CD0: {CD0:.6f}")

Final zero-lift drag coefficient, CD0: 0.033630


> __*NOTE*__: Above value of $C_{D_0}$ is higher than what was assumed (0.028) during initial weight estimation and constraint analysis

As outlined in Raymer, initial estimate for $C_{D_0}$ can also be computed using

$$
    C_{D_0} = C_{f_e} \frac{S_{wet}}{S_{ref}},
$$

where $C_{f_e}$ is the equivalent skin friction coefficient, $S_{wet}$ is total airplane wetted area, and $S_{ref}$ is the reference area. The $C_{f_e}$ is obtained from Table 12.3 and $S_{wet}$ can be easily obtained from OpenVSP model. The value obtained from this estimation can be used as a sanity check for the value obtained from detailed drag buildup method. This concludes the parasitic drag section.